# Cache and Persist

The **cache()** and **persist()** are methods used to store RDD, DataFrames and Datasets in memory to improve their re-usability across multiple Spark operations. When a dataset is `cached or persisted`, each `worker node` stores its partitioned data in memory. And `Spark’s persisted data on nodes are fault-tolerant` meaning if any partition of a Dataset is lost, it will automatically be recomputed using the original transformations that created it.

## Advantage

Computations Cost efficient - Spark computations are very expensive hence reusing the computations are used to save cost.

Computations Time efficient - Reusing the repeated computations saves lots of time.

## Dis-advantage

Large Storage Cost - As we know the memory in the worker node is shared by the computation and storage. If we persist large dataset on a worker node, the memory left for computation will be reduced. If we store the dataset on Disk, the performence will be impacted. So `don't cache/persist unless a dataset will be reused`.

## Cache vs Persist

**Cache** is the simplified version of **Persist** method. You can't specify the storage level (e.g. MEMORY_ONLY, MEMORY_ONLY_SER, MEMORY_AND_DISK). It uses the default storage level in your spark cluster config. For RDD cache() the default storage level is `MEMORY_ONLY`, for DataFrame and Dataset cache(), default is `MEMORY_AND_DISK`

```python

df.cache()
```

**persist** allows you to specify the storage level. Below is an example.

```python

from pyspark import StorageLevel
df.persist(storageLevel=StorageLevel.MEMORY_AND_DISK)
```

## Storage Level

All different storage level Spark supports are available at `org.apache.spark.storage.StorageLevel` class. The storage level specifies how and where to persist or cache a Spark DataFrame and Dataset.

- **MEMORY_ONLY**:  This is the default behavior of the RDD cache() method and stores the RDD or DataFrame as deserialized objects to JVM memory. When there is no enough memory available it will not save DataFrame of some partitions and these will be re-computed as and when required. This takes more memory. but unlike RDD, this would be slower than MEMORY_AND_DISK level as it recomputes the unsaved partitions and recomputing the in-memory columnar representation of the underlying table is expensive


- **MEMORY_ONLY_SER**:  This is the same as MEMORY_ONLY but the difference being it stores RDD as serialized objects to JVM memory. It takes lesser memory (space-efficient) then MEMORY_ONLY as it saves objects as serialized and takes an additional few more CPU cycles in order to deserialize.

- **MEMORY_ONLY_2**:  Same as MEMORY_ONLY storage level but replicate each partition to two cluster nodes.

- **MEMORY_ONLY_SER_2**:  Same as MEMORY_ONLY_SER storage level but replicate each partition to two cluster nodes.

- **MEMORY_AND_DISK**:  This is the default behavior of the DataFrame or Dataset. In this Storage Level, The DataFrame will be stored in JVM memory as a deserialized object. When required storage is greater than available memory, it stores some of the excess partitions into the disk and reads the data from the disk when required. It is slower as there is I/O involved.


- **MEMORY_AND_DISK_SER**:  This is the same as MEMORY_AND_DISK storage level difference being it serializes the DataFrame objects in memory and on disk when space is not available.

- **MEMORY_AND_DISK_2**:  Same as MEMORY_AND_DISK storage level but replicate each partition to two cluster nodes.

- **MEMORY_AND_DISK_SER_2**:  Same as MEMORY_AND_DISK_SER storage level but replicate each partition to two cluster nodes.

- **DISK_ONLY**:  In this storage level, DataFrame is stored only on disk and the CPU computation time is high as I/O is involved.

- **DISK_ONLY_2**:  Same as DISK_ONLY storage level but replicate each partition to two cluster nodes.


## Other important point

- Spark automatically monitors every persist() and cache() calls you make and `it checks usage on each node and drops persisted data if not used or using least-recently-used (LRU) algorithm`. The manually clean action **unpersist()** method can be used to.
- On Spark UI, the Storage tab shows where partitions exist in memory or disk across the cluster.
- Dataset cache() is an alias for persist(StorageLevel.MEMORY_AND_DISK)
- Caching of Spark DataFrame or Dataset is a **lazy operation**, meaning a DataFrame will not be cached until you trigger an action. 

In [1]:
from pyspark.sql import SparkSession
import os

local=True
if local:
    spark = SparkSession.builder\
        .master("local[4]")\
        .appName("CacheAndPersist")\
        .config("spark.executor.memory", "4g")\
        .getOrCreate()
else:
    spark = SparkSession.builder\
        .master("k8s://https://kubernetes.default.svc:443")\
        .appName("CacheAndPersist")\
        .config("spark.kubernetes.container.image", "inseefrlab/jupyter-datascience:master")\
        .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT'])\
        .config("spark.executor.instances", "4")\
        .config("spark.executor.memory","2g")\
        .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE'])\
        .getOrCreate()

# make the large dataframe show pretty
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

23/09/06 15:34:47 WARN Utils: Your hostname, pengfei-Virtual-Machine resolves to a loopback address: 127.0.1.1; using 10.50.2.80 instead (on interface eth0)
23/09/06 15:34:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/06 15:34:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
filePath = "/home/pengfei/data_set/sf_fire/sf_fire_snappy.parquet"

df = spark.read.parquet(filePath)

In [4]:
df.show()

+----------+------+--------------+--------------------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----------------+---------+-----------+----+----------------+--------+-------------+-------+--------------------+--------------+--------------+--------------------------+----------------------+------------------+--------------------+----------------+--------------------+
|CallNumber|UnitID|IncidentNumber|            CallType|  CallDate| WatchDate|        ReceivedDtTm|           EntryDtTm|        DispatchDtTm|        ResponseDtTm|         OnSceneDtTm|       TransportDtTm|        HospitalDtTm|CallFinalDisposition|       AvailableDtTm|             Address|         City|ZipcodeofIncident|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|       CallTypeGroup|NumberofAla

In [5]:
# now we try to cache the dataframe
# note that the cache() method is lazy transformation, if no action, it will not be executed.
cachedDf = df.cache()
cachedDf.show()

23/09/06 16:08:10 WARN BlockManager: Block rdd_18_2 could not be removed as it was not found on disk or in memory
23/09/06 16:08:10 ERROR Executor: Exception in task 1.0 in stage 8.0 (TID 18)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:61)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:348)
	at org.apache.spark.sql.execution.columnar.BasicColumnBuilder.build(ColumnBuilder.scala:80)
	at org.apache.spark.sql.execution.columnar.NativeColumnBuilder.org$apache$spark$sql$execution$columnar$NullableColumnBuilder$$super$build(ColumnBuilder.scala:98)
	at org.apache.spark.sql.execution.columnar.NullableColumnBuilder.buildNonNulls(NullableColumnBuilder.scala:86)
	at org.apache.spark.sql.execution.columnar.NullableColumnBuilder.buildNonNulls$(NullableColumnBuilder.scala:84)
	at org.apache.spark.sql.execution.columnar.NativeColumnBuilder.buildNonNulls(ColumnBuilder.scala:98)
	at org.apache.spark.sql.execution.columnar.comp

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/pengfei/.cache/pypoetry/virtualenvs/sparkcommonfunc-jby-k8HJ-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_56230/2699477237.py", line 4, in <module>
    cachedDf.show()
  File "/home/pengfei/.cache/pypoetry/virtualenvs/sparkcommonfunc-jby-k8HJ-py3.8/lib/python3.8/site-packages/pyspark/sql/dataframe.py", line 606, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/home/pengfei/.cache/pypoetry/virtualenvs/sparkcommonfunc-jby-k8HJ-py3.8/lib/python3.8/site-packages/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/home/pengfei/.cache/pypoetry/virtualenvs/sparkcommonfunc-jby-k8HJ-py3.8/lib/python3.8/site-packages/pyspark/sql/utils.py", line 190, in deco
    return f(*a, **kw)
  File "/home/pengfei/.cache/pypoetry/virtualen

ConnectionRefusedError: [Errno 111] Connection refused

The above operation failed, because we only have 4 GB memory for the worker and half of it is reserved for the 

## Performence test